# Snakemake on UA HPC Tutorial

## Why use Snakemake on HPC

+ Snakemake is a handy workflow manager written in Python. 
+ It handles workflow based on predefined job dependencies. 
+ One of the great features of Snakemake is that it can manage a workflow on both a standalone computer, or a clustered HPC system. HPC, or “cluster” as it’s often referred to, requires additional considerations.

<div class="alert alert-block alert-danger">
<b>Attention:</b> On HPC, all computing jobs should be submitted to “compute nodes” through a workload manager (for example, Slurm). Running jobs on “heaed nodes”, or log-in nodes, is a big no-no: doing so risks exhausting computing resources of head node computer (which is very limited) and slowing down everyone else on that same head node.
</div>

## Learning Objectives

**Part-1 : Learn snakemake basics in puma jupyter notebook via HPC Open On-Demand**

**Part-2 : launch interactive node on puma and create conda environment for snakemake**

**Part-3 : submit snakemake workflow using as single job on one node with 48 cpus** 

**Part-4 : submit snakemake workflow using `--cluster` submit option using `cluster_config.yml` to parallelize workflow**

## **Part-1 : Learn snakemake basics in puma jupyter notebook via HPC Open On-Demand**

Open On-Demand is a web browser interface to UArizona HPC portal. The portal handles authentication and provides direct access to UA HPC.

1. From your workstation / laptop go to: https://ood.hpc.arizona.edu (You will be asked to authenticate with dual factor).

2. From the Interactive Apps  dropdown, choose `PUMA Jupyter Notebook`.

3. Choose the Number of Hours and Number of CPU's.  The default is 1 because that will start faster in many cases. For Account enter the group that you belong to.  This is the same entry that would be in a scheduler script under "group_list". Choose the "standard" queue if possible.  "windfall" may take a lot longer. Select Launch.

4. You know your session has started by the green status lines, the status changes from Queued to Running, and when the entry next to Host has the name of the compute node you will be using. Choose `Connect to Jupyter`.

5. The next screen shows your File Manager screen.  For this tutorial, on the right dropdown choose New, `Bash notebook`.

Let us install Snakemake from the anaconda package manager. First, load the required module as shown and executing the cell.

In [ ]:
module load anaconda

and install Snakemake-minimal from the Bioconda channel as:

In [ ]:
conda install -c bioconda snakemake-minimal=5.10.0 -y

check installation by invoking snakemake help as

In [ ]:
snakemake --help

### Snakemake Basics

In this part of the tutorial we will create a very simple workflow from scratch, in order to show the fundamentals of how Snakemake works. The workflow will take two files as inputs, `a.txt` and `b.txt`, and the purpose is to convert the text in the files to upper case and then to concatenate them.

Run the following shell commands. The first one will make an empty file named `Snakefile`, which will later contain the workflow. The second and third commands generate two files containing some arbitrary text.

In [ ]:
touch Snakefile
echo "This is a.txt" > a.txt
echo "This is b.txt" > b.txt

Then open `Snakefile` by double-clicking the file in the navigation panel to your left. A Snakemake workflow is based on rules which take some file(s) as input, performs some type of operation on them, and generate some file(s) as outputs. Here is a very simple rule that takes `a.txt` as an input and produces `a.upper.txt` as an output. Copy this rule to your `Snakefile` and save it.

In [ ]:
rule convert_to_upper_case:
    input:
        "a.txt"
    output:
        "a.upper.txt"
    shell:
        """
        tr [a-z] [A-Z] < {input} > {output}
        """

<div class="alert alert-block alert-info">
<b>Tip:</b> Indentation is important in Snakefiles, so make sure that you have the correct number of spaces before input/output/shell and their respective subsections. The number of spaces per level doesn't matter as long as you're consistent. Here we use four, but you could just as well use two for a more compact look. Don't use tabs (unless your editor automatically converts them to spaces).
</div>

A rule has a name, here it's `convert_to_upper_case`. Make an effort to name your rules in a way that makes it easy to understand the purpose of the rule, as rule names are one of the main ways to interact with the workflow. The shell section contains the shell commands that will convert the text in the input file to upper case and send it to the output file.

In the shell command string, we can refer to elements of the rule via curly brackets. Here, we refer to the output file by specifying {output} and to the input file by specifying {input}. 

If you're not very familiar with Bash, this particular command can be read like "send the contents of a.txt to the program tr, which will convert all characters in the set [a-z] to the corresponding character in the set [A-Z], and then send the output to a.upper.txt"

Now let's run our first Snakemake workflow. When a workflow is executed Snakemake tries to generate a set of target files. Target files can be specified via the command line (or, as you will see later, in several other ways). Here we ask Snakemake to make the file `a.upper.txt`.

It's good practice to first run with the flag `-n`(or `--dry-run`), which will show what Snakemake plans to do without actually running anything. You can also use the flag `-p`, for showing the shell commands that it will execute, and the flag `-r` for showing the reason for running a specific rule. `snakemake --help` will show you all available flags.

In [ ]:
snakemake -n -r -p a.upper.txt

You can see that Snakemake plans to run 1 job: the rule convert_to_upper_case with `a.txt` as input and `a.upper.txt` as output. The reason for doing this is that it's missing the file `a.upper.txt`. Now execute the workflow without the `-n` flag and check that the contents of `a.upper.txt` is as expected. Then try running the same command again. What do you see? It turns out that Snakemake only reruns jobs if one of the input files is newer than one of the output files, or if one of the input files will be updated by another job. This is how Snakemake ensures that everything in the workflow is up to date. We will get back to this shortly.

What if we ask Snakemake to generate the file b.upper.txt?

In [ ]:
snakemake -n -r -p b.upper.txt

That didn't work well. We could copy the rule to make a similar one for `b.txt`, but that would be a bit cumbersome. Here is where named wildcards come in; one of the most powerful features of Snakemake. Simply change the input from input: "a.txt" to input: "{some_name}.txt" and the output to output: "{some_name}.upper.txt". Now try asking for `b.upper.txt` again.

<div class="alert alert-block alert-success">
<b>Tada!</b> What happens here is that Snakemake looks at all the rules it has available (actually only one in this case) and tries to assign values to all wildcards so that the targeted files can be generated. In this case it was quite simple, you can see that it says that wildcards: some_name=b, but for large workflows and multiple wildcards it can get much more complex. Named wildcards is what enables a workflow (or single rules) to be efficiently generalized and reused between projects or shared between people.
</div>

It seems we have the first part of our workflow working, now it's time to make the second rule for concatenating the outputs from `convert_to_upper_case`. The rule structure will be similar; the only difference is that here we have two inputs instead of one. This can be expressed in two ways, either with named inputs like this:

In [ ]:
input:
    firstFile="...",
    secondFile="..."
shell:
    """
    some_function {input.firstFile} {input.secondFile}
    """

Or with indexes like this:

In [ ]:
input:
    "...",
    "..."
shell:
    """
    some_function {input[0]} {input[1]}
    """

<div class="alert alert-block alert-danger">
<b>Attention:</b> If you have multiple inputs or outputs they need to be delimited with a comma (as seen above). This is a very common mistake when writing Snakemake workflows. The parser will complain, but sometimes the error message can be difficult to interpret.
</div>

Now try to construct this rule yourself and name it `concatenate_a_and_b`. The syntax for concatenating two files in Bash is `cat first_file second_file > output_file`. Call the output `c.txt`. Run the workflow in Snakemake and validate that the output looks as expected.

Wouldn't it be nice if our workflow could be used for any files, not just `a.txt` and `b.txt`? We can achieve this by using named wildcards (or in other ways as we will discuss later). As we've mentioned, Snakemake looks at all the rules it has available and tries to assign values to all wildcards so that the targeted files can be generated. We therefore have to name the output file in a way so that it also contains information about which input files it should be based on. Try to figure out how to do this yourself. If you're stuck you can look at the spoiler below, but spend some time on it before you look. Also rename the rule to `concatenate_files` to reflect its new more general use.

In [ ]:
rule convert_to_upper_case:
    input:
        "{some_name}.txt"
    output:
        "{some_name}.upper.txt"
    shell:
        """
        tr [a-z] [A-Z] < {input} > {output}
        """

rule concatenate_files:
    input:
        "{first}.upper.txt",
        "{second}.upper.txt"
    output:
        "{first}_{second}.txt"
    shell:
        """
        cat {input[0]} {input[1]} > {output}
        """

We can now control which input files to use by the name of the file we ask Snakemake to generate.

In [ ]:
snakemake a_b.txt

Nice!

<div class="alert alert-block alert-info">
<b>Quick Recap:</b> In this section we've learned:

    How a simple Snakemake rule looks.
    How to define target files when executing a workflow.
    How to use named wildcards for writing generic and flexible rules.

</div>

#### Visualizing workflows

All that we've done so far could quite easily be done in a simple shell script that takes the input files as parameters. Let's now take a look at some of the features where a WMS like Snakemake really adds value compared to a more straightforward approach. One such feature is the possibility to visualize your workflow. Snakemake can generate two types of graphs, one that show how the rules are connected and one that shows how the jobs (i.e. an execution of a rule with some given inputs/outputs/settings) are connected. First we look at the rule graph. The following command will generate a rule graph in the dot language and pipe it to the program dot, which in turn will save a visualization of the graph as a PNG file (if you're having troubles displaying PNG files you could use SVG or JPG instead).

In [ ]:
snakemake --rulegraph a_b.txt | dot -Tpng > rulegraph.png

The second type of graph is based on the jobs, and looks like this for our little workflow (use --dag instead of --rulegraph).

In [ ]:
snakemake --dag a_b.txt | dot -Tpng > jobgraph.png

The main difference here is that now each node is a job instead of a rule. You can see that the wildcards used in each job are also displayed. Another difference is the dotted lines around the nodes. A dotted line is Snakemake's way of indicating that this rule doesn't need to be rerun in order to generate `a_b.txt`. Validate this by running `snakemake -n -r a_b.txt` and it should say that there is nothing to be done.

We've discussed before that one of the main purposes of using a WMS is that it automatically makes sure that everything is up to date. This is done by recursively checking that outputs are always newer than inputs for all the rules involved in the generation of your target files. Now try to change the contents of `a.txt` to some other text and save it. What do you think will happen if you run `snakemake -n -r a_b.txt` again?

In [ ]:
snakemake -n -r a_b.txt

Were you correct? Also generate the job graph and compare to the one generated above. What's the difference? Now rerun without -n and validate that a_b.txt contains the new text. Note that Snakemake doesn't look at the contents of files when trying to determine what has changed, only at the timestamp for when they were last modified.

We've seen that Snakemake keeps track of if files in the workflow have changed, and automatically makes sure that any results depending on such files are regenerated. What about if the rules themselves are changed? It turns out that there are multiple ways to do this, but the most straightforward is to manually specify that you want to rerun a rule (and thereby also all the steps between that rule and your target). Let's say that we want to modify the rule concatenate_files to also include which files were concatenated.

In [ ]:
rule concatenate_files:
    input:
        "{first}.upper.txt",
        "{second}.upper.txt"
    output:
        "{first}_{second}.txt"
    shell:
        """
        echo 'Concatenating {input}' | cat - {input[0]} {input[1]} > {output}
        """

If you now run the workflow as before you should get "Nothing to be done", because no files involved in the workflow have been changed. Instead we have to force Snakemake to rerun the rule by using the `-R` flag. Let's try a dry-run.

In [ ]:
snakemake a_b.txt -r -n -R concatenate_files

Note that the reason for the job is now "Forced execution". You can target files as well as rules, so you would get the same result with `-R a_b.txt`. Whenever you've made changes to a rule that will affect the output it's good practice to force re-execution like this. Still, there can be situations where you don't know if any rules have been changed. Maybe several people collaborate on the same workflow but are using it on different files, for example. Snakemake keeps track of how all files were generated (when, by which rule, which version of the rule, and by which commands). You can export this information to a tab-delimited file like this:

In [ ]:
snakemake a_b.txt -D > summary.tsv

The contents of summary.tsv is shown in the table below (scroll horizontally to see the full table).

You can see in the second last column that the rule implementation for a_b.txt has changed. The last column shows if Snakemake plans to regenerate the files when it's next executed. None of the files will be regenerated because Snakemake doesn't regenerate files by default if the rule implementation changes. From a reproducibility perspective maybe it would be better if this was done automatically, but it would be very computationally expensive and cumbersome if you had to rerun your whole workflow every time you fix a spelling mistake in a comment somewhere. So, it's up to us to look at the summary table and rerun things as needed. You can get a list of the files for which the rule implementation has changed, and then force Snakemake to regenerate these files with the -R flag.

You might wonder where Snakemake keeps track of all these things? It stores all information in a hidden subdirectory called `.snakemake`. This is convenient since it's easy to delete if you don't need it anymore and everything is contained in the project directory. Just be sure to add it to .gitignore so that you don't end up tracking it.

<div class="alert alert-block alert-info">
<b>Quick Recap:</b> In this section we've learned:

    How to use --dag and --rulegraph for visualizing the job and rule graphs, respectively.
    How to force Snakemake to rerun relevant parts of the workflow after there have been changes.
    How logging in Snakemake works.

</div>

## **Part-2 : launch interactive node on puma and create conda environment for snakemake**

Logging in to UA HPC

In [ ]:
ssh <netid>@hpc.arizona.edu

After connecting to the bastion host, you will see:

In [ ]:
Success. Logging you in...
Last login:
This is a bastion host used to access the rest of the RT/HPC environment.
 
Shortcut commands to access each resource
-----------------------------------------
Ocelote:
$ ocelote
 
El Gato:
$ elgato
 
Puma:
$ puma

Type puma and press enter to connect.

#### Clone git repository for Snakemake workflow 

In [ ]:
git clone https://github.com/sateeshperi/UAHPC_Snakemake
cd UAHPC_Snakemake
ls

This repository contains an `environment.yml` file that is used to install all the necessary tools for RNAseq snakemake workflow

Let's build a conda environment for Snakemake using an interactive node on Puma. Launch an inode by typing:

In [ ]:
interactive

The change in the command line prompt shows you are now on a compute node. From there, you may view and access individual software packages which are made available as modules. Modules allow for the customization of your environment, including allowing access to different versions of the same software. Take a look at what’s installed by running the following command:

In [ ]:
module avail

module load anaconda

Now create a conda environment using the environment.yml file:

In [ ]:
conda env create -n snakemake -f environment.yml

Activate the environment by:

In [ ]:
source activate snakemake

By default, Conda will add information to your prompt telling you which environment that is active.

check Snakemake isntallation and version by:

In [ ]:
snakemake --version

To see all your environments you can run:

In [ ]:
conda info --envs

The active environment will be marked with an asterisk.

Conda Environments are available for all slurm jobs submitted and now that we have setup our conda environments for our Snakemake workflow, we can exit the interactive node and submit using slurm scheduler. to exit the interactive node type:

In [ ]:
exit

Instead of reserving an interactive job and running snakemake inside that job, we want to use snakemake's cluster functionality to make it submit jobs to Slurm. 

## **Part-3 : submit snakemake workflow using as single job on one node with 48 cpus** 

In [ ]:
#!/bin/bash -login
#SBATCH -J snakemake
#SBATCH --account=bjoyce3
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH -N 1
#SBATCH -c 48


# activate conda in general
module load anaconda

# activate a specific conda environment, if you so choose
source activate smake_rnaseq

# go to a particular directory
cd /home/u15/sateeshp/snakemake_rnaseq

# make things fail on errors
set -o nounset
set -o errexit
set -x

### run your commands here!
snakemake

In [ ]:
qstat -u <username>

In [ ]:
junonia.hpc.arizona.edu:
                                                                               Req'd  Req'd   Elap 
Job id               Username Queue    Name                 SessID NDS   TSK   Memory Time Use S Time 
-------------------- -------- -------- -------------------- ------ ----- ----- ------ ----- - -----
318195               sateeshp standard snakemake            --         1    48     -- 01:00 Q 00:00

## **Part-4 : submit snakemake workflow using `--cluster` submit option using `cluster_config.yml` to parallelize workflow**

### Configure job parameters with cluster.yaml

Snakemake can be taught to submit batch jobs by providing a template string that has access to many of the properties of jobs (threads, resources, params) specified in the Snakefile, as well as parameters defined in a cluster config file. As for the general config file, the cluster config file can be in yaml or json format.

Options under the __default__ header apply to all rules, but it's possible to override them selectively with rule-specific options.

Create the file cluster.yaml in the same directory as the Snakefile with `nano cluster.yaml` and paste the following content

In [ ]:
# cluster_config.yml - cluster configuration
__default__:
    account: "bjoyce3"
    partition: "standard"
    time: "00:20:00" # time limit for each job
    cpus: "4" #Request n cores be allocated per node.
    mem: "4g"
    output: "logs_slurm/{rule}.{wildcards}.out"
    error: "logs_slurm/{rule}.{wildcards}.err"

It makes sense to use this file for cluster specific settings such as partitions and local scratch and runtimes. This file can contain a special `__default__` entry whose values will be used for any rules that do not explicitly specify values for the keys in question. In this example, hisat would be run on the norm partition with a walltime of 10 min and 10GB of lscratch - the latter two from the `__default__` section since they are not defined for the hisat2 rule.

some settings you may need to change per rule are:

    time, to adjust to the runtime of the rule,

    cpus, to adjust to the number of threads specified in the rule,

    partition, to go to bigmem nodes, for example,


When submitting to the cluster, a number of other of other options are important:

`-k`, `--keep-going`: By default, snakemake will quit if a job fails (after waiting for running jobs to finish. -k will make snakemake continue with independent jobs.

`-w`, `--latency-wait`, `--output-wait`: The amount of time snakemake will wait for output files to appear after a job has finished. This defaults to a low 5s. On the shared file systems latency may be higher. Raising it to 120s is a bit excessive, but it doesn't really hurt too much.

`--local-cores`: The number of CPUs available for local rules
    
`--max-jobs-per-second`: Max numbers of jobs to submit per second. Please be kind to the batch scheduler.

`--cluster`: The template string used to submit each (non local) job.

`--jobs`: The number of jobs to run concurrently.
    
`--cluster-config`: The cluster config file


as long as the software you use only does multithreading or doesn't scale at all. With multithreading you cannot run across multiple nodes, so nodes needs to be 1 , and snakemake always just runs one task per job, so ntasks also stays at 1.

<div class="alert alert-block alert-danger">
<b>Attention:</b> Be aware that ncpus should match the threads directive in the respective rule. If ncpus is less than threads snakemake will reserve only ncpus cores, but run the rule on the number of threads specified with threads . When running on the Iris cluster, the value for both should not exceed 28, the number of cores on a regular node.
</div> 

In [ ]:
#!/bin/bash -login
#SBATCH -J snakemake
#SBATCH --account=bjoyce3
#SBATCH --partition=standard
#SBATCH -t 01:00:00
#SBATCH -N 1


# activate conda in general
module load anaconda

# activate a specific conda environment, if you so choose
source activate smake_rnaseq

# go to a particular directory
cd /home/u15/sateeshp/snakemake_rnaseq

# make things fail on errors
set -o nounset
set -o errexit
set -x

### run your commands here!
snakemake -s Snakefile --cluster-config cluster.yaml --cluster "sbatch --account={cluster.account} --partition={cluster.partition} --time={cluster.time} --cpus-per-task={cluster.cpus} -J snake --mem={cluster.mem} -o {cluster.output} -e {cluster.output}" --jobs 5 --latency-wait=15

And then submit this script to cluster using `sbatch scheduler.sh`. Then all you need to do is sit back and wait for it to finish (or error out).

In the above command, we use `--cluster` to hand Snakemake a command that it can use to submit a job script. Snakemake creates a job script for each job to be run and uses this command to submit that job to HPC. That simple! Notice that here we use sbatch because it’s the command you would use to submit a shell script on HPC managed by Slurm. Replace this with qsub commands if your HPC uses SGE. -j 10 argument limits to at most 10 jobs running at the same time.

Check the status of submitted jobs by:

In [ ]:
qstat -u <username>

In [ ]:
junonia.hpc.arizona.edu:
                                                                               Req'd  Req'd   Elap 
Job id               Username Queue    Name                 SessID NDS   TSK   Memory Time Use S Time 
-------------------- -------- -------- -------------------- ------ ----- ----- ------ ----- - -----
318196               sateeshp standard snakemake            --         1     1     -- 02:00 R 00:00
318197               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00
318198               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00
318199               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00
318200               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00
318201               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00
318202               sateeshp standard snake                --         1     4     -- 00:20 Q 00:00

<div class="alert alert-block alert-success">
<b>Useful stuff:</b> To avoid too much overhead in the number of jobs submitted to Slurm, use thegroup directive to group rules that can run together in a single job.
</div>